# ML Final Project

## Goals of the Project

The project studes and attempts to quantify the predictability of the Bundesliga football games based on the information contained in game statistics staring from 2006 onwards. In particular,

* we use a deep neural network to examine the correlations between different features provided from the statistics of individual games
* we ask the question to what accuracy one can determine the outcome and score of any Bundesliga game, given the game statistics.
* based on our model, we compare the predictions of our model on the Bundesliga season 2016/17 up to present date. We also and make a prediction for final Bundesliga table for season 2016/17.

Studying the predictive power on footbal statistics on the outcome of games is an interesting and challenging problem of importance both to sports in general, and the development of a proper intuition about the inner workings of machine learning techniques. The difficulty of the problem is enhanced by the occurence of outlier events in the data, for example a team performing better in a given game but nevertheless losing to their opponent due to 'luck'. These events can easily be recognised as noise or bias by the neural network leading to an increase of both the in-sample and out-of-sample error. 


## Data

We use the football games dataset provided at ... We then modified the set introducing some new features as combinations of the existing ones, while dropping others.

Each data point is represented as a vector containing the following features:

Last, each feature from the data was normalised to unity with respect to all games in the data set.

## Model: Deep Neural Network

Our model is a deep neural net, consisting of an input layer, four hidden layers and an output layers:
* input layer: ?? features
* hidden layer 1: 50 neurons
* hidden layer 2: 500 neurons
* hidden layer 3: 500 neurons
* hidden layer 4: 50 neurons
* output layer with softmax activation: ?? features

We train with minibatches using 'adam', and minimise the categorical entropy function. Additionally, we apply 'dropout' regularisation after hidden layer 4. 

here comes the code...

## Results

show some plots here